In [40]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
plt.rc('font', size=10)
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0)
from time import time
import pickle
import gc
!pip install iisignature
import iisignature
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/My Drive/Colab Notebooks/Thesis'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Train

##Define model structure.

In [41]:
np.random.seed(42)
torch.manual_seed(42)

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [43]:
class fBM:
    def __init__(self, H, M, T, N, d, device=device):

        self.H = H # Hurst parameter
        self.T = T # time horizon
        self.N = N # number of time steps
        self.M = M # number of paths
        self.d = d # dimension of state, length of historical window
        self.device = device

        self.t = torch.linspace(0, self.T, self.N + 1).to(device) # shape (N+1, )
        self.paths = self.simulate() # shape (M, N+1)

        # if run out of RAM, add additional argument 'method' and initialise corresponding matrix
        self.X = self.get_Markov_process() # shape (M, N+1, d)
        self.signatures = self.get_signature()
        self.randomised_signatures = self.compute_randomised_signatures()

    def gamma(self, k):
        return 0.5 * ((k + 1)**(2 * self.H) + np.abs(k - 1)**(2 * self.H) - 2 * k**(2 * self.H))

    def get_L(self):
        # Cholesky decomposition of the covariance matrix
        L = torch.zeros((self.N, self.N), dtype=torch.float32).to(self.device)
        L[0,0] = 1.0
        L[1,0] = self.gamma(1)
        L[1,1] = torch.sqrt(1 - (L[1,0]**2))
        for i in range(2,self.N):
            L[i,0] = self.gamma(i)
            for j in range(1, i):
                L[i,j] = (1/L[j,j])*(self.gamma(i-j) - (L[i,0:j] @ L[j,0:j]))

            L[i,i] = torch.sqrt(1 - torch.sum((L[i,0:i]**2)))

        return L

    def simulate(self):
        # simulate M fBM paths using Cholesky decomposition
        # return shape: (M, N+1)
        L = self.get_L()
        V = torch.randn((self.M, self.N), dtype=torch.float32).to(self.device)
        X = torch.matmul(V, L.T)  # L.T to align with the row-wise multiplication

        # Compute cumulative sum and scaling
        tem = torch.cumsum(X, dim=1) * (self.N ** -self.H)
        tem = torch.cat([torch.zeros((self.M, 1), dtype=torch.float32).to(self.device), tem], dim=1)
        tem = (self.T ** self.H) * tem

        return tem

    def plot(self):
        plt.figure(figsize=(5, 3))
        for m in range(self.M):
            plt.plot(self.t, self.paths[m, :], label=f'H = {self.H}')
        plt.title('fBM')
        plt.xlabel("Time")
        plt.ylabel("Paths")
        plt.show()

    def get_Markov_process(self):
        # return X of shape: (M, N+1, d)
        X = torch.zeros((self.M, self.N + 1, self.d), dtype=torch.float32).to(self.device)

        for n in range(1, self.N + 1):
            end_idx = min(n, self.d)
            X[:, n, 0:end_idx] = torch.flip(self.paths[:, 1:(end_idx+1)], dims=(1,))

        return X

    def get_time_augmented_path(self):
        # return time augmented fBM of shape: (M, N+1, 2) where [:, :, 0] corresponds to the time
        B_hat = torch.zeros((self.M, self.N + 1, 2), dtype=torch.float32).to(self.device)
        t = torch.linspace(0, self.T, self.N + 1)
        B_hat[:, :, 0] = t
        B_hat[:, :, 1] = self.paths

        return B_hat

    def get_signature(self, truncation_level=3):
        # Return signatures of shape: (M, N+1, siglength)
        # where siglength = 2 + ... + 2^truncation_level.
        B_hat = self.get_time_augmented_path().cpu().numpy()
        sig = np.zeros((self.M, self.N + 1, iisignature.siglength(2, truncation_level)))
        sig[:, 0, 0] = 1
        for i in range(1, self.N + 1):
            sig[:, i, :] = iisignature.sigjoin(sig[:, i - 1, :], B_hat[:, i] - B_hat[:, i - 1], truncation_level)

        sig = torch.tensor(sig).to(self.device)

        return sig

    def compute_randomised_signatures(self, k=3):
        """
        paths: shape (M, N+1, 2)
        k: dimension of randomised signatures
        sigma: activation function
        return: shape (M, N+1, k)
        """
        sigma = nn.ReLU()

        B_hat = self.get_time_augmented_path()

        Z = torch.zeros((self.M, self.N + 1, k), dtype=torch.float32).to(device)
        Z[:, 0, :] = torch.randn((self.M, k), dtype=torch.float32).to(device)

        A = torch.randn(size=(self.d, k, k), dtype=torch.float32).to(device)
        b = torch.randn(size=(self.d, k), dtype=torch.float32).to(device)

        for m in range(self.M):
            for n in range(1, self.N + 1):
                tem = 0
                for i in range(2):
                   tem += (sigma(A[i, :, :]) @ Z[m, n - 1, :] + b[i, :]) * (B_hat[m, n, i] - B_hat[m, n - 1, i])
                Z[m, n, :] = Z[m, n - 1, :] + tem

        return Z

    def g(self, m, n):
        return self.X[m, n, 0]

In [44]:
class FNN(torch.nn.Module):
    def __init__(self, input_dimension, q1=140, q2=140):
        super(FNN, self).__init__()
        self.a1 = nn.Linear(input_dimension, q1)
        self.relu = nn.ReLU()
        self.a2 = nn.Linear(q1, q2)
        self.a3 = nn.Linear(q2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.a1(x)
        out = self.relu(out)
        out = self.a2(out)
        out = self.relu(out)
        out = self.a3(out)
        out = self.sigmoid(out)
        return out

In [45]:
def score(tau_mat, fBM, s=0.5):
    paths = fBM.paths # (M, N+1)

    true_ost = paths.argmax(dim=1)
    true_sup = paths.gather(1, true_ost.unsqueeze(1)).squeeze(1)  # Gather true_sup values

    true_ost_mean = true_ost.float().mean()
    true_sup_mean = true_sup.mean()

    tau_mat_ = torch.from_numpy(tau_mat).to(device)

    approx_ost = tau_mat_[:, 0].long()  # Ensure it's a long tensor for indexing
    approx_sup = paths.gather(1, approx_ost.unsqueeze(1)).squeeze(1)  # Gather approx_sup values

    ost_diff = torch.abs(true_ost - approx_ost).float()  # Difference between true_ost and approx_ost
    sup_diff = torch.abs(true_sup - approx_sup)  # Difference between true_sup and approx_sup

    ost_diff_mean = ost_diff.mean()
    sup_diff_mean = sup_diff.mean()

    return s * ost_diff_mean / true_ost_mean + (1 - s) * sup_diff_mean / true_sup_mean

In [46]:
def loss(y_pred, my_fBM, X_train_batch, n, tau_n_plus_one, method):
    """
    y_pred: predicted probability to stop at n-th time step
    size = batch_size when trained on batched;
         = M_train when evaluated on full set.
    """
    # averaged loss at n-th time step
    size = X_train_batch.shape[0]
    loss_n = torch.zeros(size)
    for m in range(0, size):
        # loss for m-th path at n-th time step is minus immediate reward
        loss_n[m] = - my_fBM.g(m, n) * y_pred[m] - my_fBM.g(m, int(tau_n_plus_one[m])) * (1 - y_pred[m])

    return loss_n.mean()

In [47]:
def weights_init(model, init_type='xavier', **init_kwargs):
    """
    Initialise the weights of a PyTorch model.

    Parameters:
        model (nn.Module): The PyTorch model to initialise.
        init_type (str, optional): The type of weight initialisation to use.
            Must be either 'xavier' or 'kaiming'.
            Default is 'xavier'.

    Raises:
        ValueError: If the init_type provided is not valid.
    """
    if init_type == 'xavier':
        initializer = nn.init.xavier_normal_
    elif init_type == 'kaiming':
        initializer = nn.init.kaiming_normal_
    else:
        raise ValueError(f'Invalid initialization type: {init_type}')

    def init_func(m):
        if isinstance(m, nn.Linear):
            initializer(m.weight.data, **init_kwargs)
            if m.bias is not None:
                nn.init.constant_(m.bias.data, 0)

    model.apply(init_func)

In [48]:
def train(n, my_fBM, X_train, tau_n_plus_one, method, rc=False):
    """
    n: current time step
    tau_n_plus_one: stopping time at (n+1)-th time step
    F_n: shape (fBMs, 1)
    rc: reservoir computing
    """
    num_epochs = 20
    max_lr = 0.01 # may be too large, try 0.001, 0.0001
    batch_size = 128 # the higher, the smaller the error in MC, usually 64 < batch_size < 512, power of 2

    train_data = TensorDataset(X_train)

    dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

    if method == "DPO" or "Deep_Signature_Stopping":
        model = FNN(input_dimension=X_train.shape[-1]).to(device)

    if method == "Randomised_Signature_Stopping":
        model = FNN(input_dimension=X_train.shape[-1]).to(device)
        weights_init(model)
        for name, param in model.named_parameters():
            if 'a3' not in name:
                param.requires_grad = False

    optimizer = torch.optim.Adam(model.parameters(), lr=max_lr)

    # decreases the learning rate by a factor of 0.1 after every epoch
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, total_steps=int(X_train.shape[0]), epochs=100)

    # batch training
    for epoch in tqdm(range(num_epochs), desc=f"Training with {num_epochs} epochs", leave=False):
        for batch in dataloader:
            X_train_batch = batch[0].float().to(device) # Each batch has shape (batch_size, N+1, d)

            F_n_batch = model.forward(X_train_batch[:, n, :].float()) # shape (batch_size, 1)
            criterion = loss(F_n_batch, my_fBM, X_train_batch, n, tau_n_plus_one, method)

            optimizer.zero_grad() # Sets all gradient fields to zero
            criterion.backward() # Populates the gradient fields with fresh values
            optimizer.step() # Performs the gradient descent step based on the values of the gradient fields

        #scheduler.step()  # Decreases the learning rate by a factor of 0.1

    # evaluate on full set to get F_n
    with torch.no_grad():  # Turn off gradient computations
        F_n = model(X_train[:, n, :].float())

    return F_n, model

In [49]:
def perform(my_fBM_train, my_fBM_test, method, H, M_train, M_test, T, N, d):
    """
    DPO: data = fBM.X
    Deep Signature Stopping: data = fBM.signatures
    Randomised Signature Stopping: data = fBM.randomised_signatures
    """
    # -------------- populate X_train --------------
    if method == "DPO":
        X_train = my_fBM_train.X
        X_test = my_fBM_test.X

    elif method == "Deep_Signature_Stopping":
        X_train = my_fBM_train.signatures
        X_test = my_fBM_test.signatures

    elif method == "Randomised_Signature_Stopping":
        X_train = my_fBM_train.randomised_signatures
        X_test = my_fBM_test.randomised_signatures

    else:
        raise NotImplementedError("Method must be 'DPO', 'Deep_Signature_Stopping' or 'Randomised_Signature_Stopping'.")

    # ---------------- training loop ----------------
    start = time() # start timing

    models = [None] * N # store trained models

    tau_mat_train = np.zeros((M_train, N+1))
    tau_mat_train[:, -1] = N # need to divide by N later

    f_mat_train = np.zeros((M_train, N+1))
    f_mat_train[:, -1] = 1

    V_mat_train = np.zeros((M_train, N+1))
    V_est_train = np.zeros(N+1)

    for m in range(0, M_train):
        V_mat_train[m, N] = my_fBM_train.g(m, N)

    V_est_train[N] = np.mean(V_mat_train[:, N])

    for n in range(N-1, -1, -1):
        print(f"----{n}----")
        F_n, model_n = train(n, my_fBM_train, X_train, torch.from_numpy(tau_mat_train[:, n+1]).float(), method)
        models[n] = model_n
        n_probs = F_n.detach().cpu().numpy().reshape(M_train)
        #print(f"{n}-th time step: min {np.min(n_probs)}, max {np.max(n_probs)}")
        f_mat_train[:, n] = (n_probs > 0.5) * 1.0 # (0, 1) -> {0, 1}
        tau_mat_train[:, n] = np.argmax(f_mat_train, axis=1) # first occurence of 1, need to divide by N later

        for m in range(0, M_train):
            V_mat_train[m, n] = my_fBM_train.g(m, int(tau_mat_train[m, n]))

    stop = time() # stopping timing

    score_train = score(tau_mat_train, my_fBM_train)
    tau_mat_train /= N

    V_est_train = np.mean(V_mat_train, axis=0)

    # ---------------- testing loop ----------------
    tau_mat_test = np.zeros((M_test, N+1))
    tau_mat_test[:, -1] = N # need to divide by N later

    f_mat_test = np.zeros((M_test, N+1))
    f_mat_test[:, -1] = 1

    V_mat_test = np.zeros((M_test, N+1))
    V_est_test = np.zeros(N+1)

    for m in range(0, M_test):
        V_mat_test[m, N] = my_fBM_test.g(m, N)

    V_est_test[N] = np.mean(V_mat_test[:, N])

    for n in range(N-1, -1, -1):
        model_n = models[n]
        F_n = model_n(X_test[:, n].float())
        n_probs = F_n.detach().cpu().numpy().reshape(M_test)
        f_mat_test[:, n] = (n_probs > 0.5) * 1.0
        tau_mat_test[:, n] = np.argmax(f_mat_test, axis=1)

        for m in range(0, M_test):
            V_mat_test[m, n] = my_fBM_test.g(m, int(tau_mat_test[m, n]))

    score_test = score(tau_mat_test, my_fBM_test)
    tau_mat_test /= N

    V_est_test = np.mean(V_mat_test, axis=0)
    V_std_test = np.std(V_mat_test, axis=0)
    V_se_test = V_std_test/(np.sqrt(M_test))

    z = scipy.stats.norm.ppf(0.975)
    lower = V_est_test[0] - z * V_se_test[0]
    upper = V_est_test[0] + z * V_se_test[0]

    # V_est[0] is the approximated supremum
    return tau_mat_train, V_est_train, score_train.detach().cpu().numpy(), tau_mat_test, lower, V_est_test, upper, score_test.detach().cpu().numpy(), stop - start

##Simulate fBM paths and save to pickles.

In [50]:
T = 1
N = 100
M_train = int(1e4)
M_test = int(1e4) # the higher, the smaller the MC error
d = 100

H_list = [0.05 * i for i in range(1, 20)]
H_list.append(0.99)
H_list.insert(0, 0.01)

# simulate fBM paths
#for H in tqdm(H_list):
#    my_fBM_train = fBM(H=H, M=M_train, T=T, N=N, d=d)
#    my_fBM_test = fBM(H=H, M=M_test, T=T, N=N, d=d)c
#    with open(PATH + f'/fBM/fBM_{H:.2f}_train.pkl', 'wb') as f:
#        pickle.dump(my_fBM_train, f)
#    with open(PATH + f'/fBM/fBM_{H:.2f}_test.pkl', 'wb') as g:
#        pickle.dump(my_fBM_test, g)

##Train models and save to pickles.

In [ ]:
for H in tqdm(H_list, desc="H"):
    with open(PATH + f'/fBM_M10000_d100/fBM_{H:.2f}_train.pkl', 'rb') as f:
        my_fBM_train = pickle.load(f)
    with open(PATH + f'/fBM_M10000_d100/fBM_{H:.2f}_test.pkl', 'rb') as g:
        my_fBM_test = pickle.load(g)
    for method in ["Deep_Signature_Stopping", "Randomised_Signature_Stopping"]:
        temp = perform(my_fBM_train, my_fBM_test, method, H, M_train, M_test, T, N, d)
        temp_data = {"H": {H}, "method": method, "tau_mat_train": temp[0], "V_est_train": temp[1], "score_train": temp[2], "tau_mat_test": temp[3], "lower": temp[4], "V_est_test": temp[5], "upper": temp[6], "score_test": temp[7], "time": temp[8]}
        with open(PATH + f'/Data/my_data_{H:.2f}_{method}.pkl', 'wb') as f:
            pickle.dump(temp_data, f)

H:   0%|          | 0/21 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.Byt

----99----


----98----


----97----


----96----


...

----81----


----80----


----79----


----78----


Training with 20 epochs:  25%|██▌       | 5/20 [00:16<00:48,  3.23s/it]

#Plot

In [ ]:
scores_train = {"DPO": [], "Deep_Signature_Stopping": [], "Randomised_Signature_Stopping": []}
scores_test = {"DPO": [], "Deep_Signature_Stopping": [], "Randomised_Signature_Stopping": []}
tau_train = {"DPO": [], "Deep_Signature_Stopping": [], "Randomised_Signature_Stopping": []}
tau_test = {"DPO": [], "Deep_Signature_Stopping": [], "Randomised_Signature_Stopping": []}
supremums_train = {"DPO": [], "Deep_Signature_Stopping": [], "Randomised_Signature_Stopping": []}
supremums_test = {"DPO": [], "Deep_Signature_Stopping": [], "Randomised_Signature_Stopping": []}
times = {"DPO": [], "Deep_Signature_Stopping": [], "Randomised_Signature_Stopping": []}

for method in tqdm(["DPO", "Deep_Signature_Stopping", "Randomised_Signature_Stopping"]):
    for H in tqdm(H_list):
        with open(PATH + f'/Data_/my_data_{H:.2f}_{method}.pkl', 'rb') as f:
            if device.type == "cpu":
                res = torch.load(f, map_location=torch.device('cpu'))
            elif device.type == "cuda":
                res = pickle.load(f)
            else:
                pass
            tau_train[f"{method}"].append(res["tau_mat_train"])
            tau_test[f"{method}"].append(res["tau_mat_test"])
            scores_train[f"{method}"].append(res["score_train"])
            scores_test[f"{method}"].append(res["score_test"])
            supremums_train[f"{method}"].append(res["V_est_train"][0])
            supremums_test[f"{method}"].append(res["V_est_test"][0])
            times[f"{method}"].append(res["time"])

In [ ]:
# convert lists of tensors to numpy arrays
scores_train_np = {}
for method in ["DPO", "Deep_Signature_Stopping", "Randomised_Signature_Stopping"]:
    scores_train_np[f"{method}"] = np.zeros(len(H_list))
    for i in range(len(H_list)):
        scores_train_np[f"{method}"][i] = scores_train[f"{method}"][i].detach().cpu().numpy()

# convert lists of tensors to numpy arrays
scores_test_np = {}
for method in ["DPO", "Deep_Signature_Stopping", "Randomised_Signature_Stopping"]:
    scores_test_np[f"{method}"] = np.zeros(len(H_list))
    for i in range(len(H_list)):
        scores_test_np[f"{method}"][i] = scores_test[f"{method}"][i].detach().cpu().numpy()

In [ ]:
# training score
fig, ax = plt.subplots(1,1)
ax.plot(H_list, scores_train_np["DPO"], label="Deep optimal stopping")
ax.plot(H_list, scores_train_np["Deep_Signature_Stopping"], label="Deep signature stopping")
ax.plot(H_list, scores_train_np["Randomised_Signature_Stopping"], label="Randomised signature stopping")
ax.set_xlabel("Hurst parameter")
ax.set_ylabel("Loss")
ax.legend()
plt.savefig(PATH + "/Plots/train_score.png")

# testing score
fig, ax = plt.subplots(1,1)
ax.plot(H_list, scores_test_np["DPO"], label="Deep optimal stopping")
ax.plot(H_list, scores_test_np["Deep_Signature_Stopping"], label="Deep signature stopping")
ax.plot(H_list, scores_test_np["Randomised_Signature_Stopping"], label="Randomised signature stopping")
ax.set_xlabel("Hurst parameter")
ax.set_ylabel("Loss")
ax.legend()
plt.savefig(PATH + "/Plots/test_score.png")

# training supremum
fig, ax = plt.subplots(1,1)
ax.plot(H_list, supremums_train["DPO"], label="Deep optimal stopping")
ax.plot(H_list, supremums_train["Deep_Signature_Stopping"], label="Deep signature stopping")
ax.plot(H_list, supremums_train["Randomised_Signature_Stopping"], label="Randomised signature stopping")
ax.set_xlabel("Hurst parameter")
ax.set_ylabel("Estimate of Supremum")
ax.legend()
plt.savefig(PATH + "/Plots/train_supremum.png")

# testing supremum
fig, ax = plt.subplots(1,1)
ax.plot(H_list, supremums_test["DPO"], label="Deep optimal stopping")
ax.plot(H_list, supremums_test["Deep_Signature_Stopping"], label="Deep signature stopping")
ax.plot(H_list, supremums_test["Randomised_Signature_Stopping"], label="Randomised signature stopping")
ax.set_xlabel("Hurst parameter")
ax.set_ylabel("Estimate of Supremum")
ax.legend()
plt.savefig(PATH + "/Plots/test_supremum.png")


In [ ]:
supremums_test

In [ ]:
scores_test_np

In [ ]:
times